# <img src="./images/SQLIcon.png?modified=23342" width=80px height=80px style="vertical-align: middle;">  SQL Best Practices


In [ ]:
#@title ### Run the following cell to download the necessary files for this lesson { display-mode: "form" } 
#@markdown Don't worry about what's in this collapsed cell

!pip install -q ipython
print('Downloading video_player.py...')
!wget https://s3-eu-west-1.amazonaws.com/aicore-portal-public-prod-307050600709/lesson_files/baa028d3-4b66-4d49-8de7-fdfd1039b2c2/video_player.py -q -O video_player.py
import video_player
print('Downloading aliasing_columns.mp4...')
!wget https://s3-eu-west-1.amazonaws.com/aicore-portal-public-prod-307050600709/lesson_files/baa028d3-4b66-4d49-8de7-fdfd1039b2c2/aliasing_columns.mp4 -q -O aliasing_columns.mp4
print('Downloading not_selecting_all_columns.mp4...')
!wget https://s3-eu-west-1.amazonaws.com/aicore-portal-public-prod-307050600709/lesson_files/baa028d3-4b66-4d49-8de7-fdfd1039b2c2/not_selecting_all_columns.mp4 -q -O not_selecting_all_columns.mp4


Following best practices when writing SQL code can help to make your code more readable, performant and less error prone. Outlined in the notebook will be some best practices to keep in mind when writing SQL code. First a few things to know about writing SQL queries:

>- SQL Statements terminate with a semi-colon `;`
>- SQL ignores whitespace
>- `--` can be used for single line comments and `/* */` can be used for multiline comments, everything between  `/* */` will be treated as comments


## Best Practices when writing SQL code

The Pagila database will be used in all cases here so make sure you connect to it if you want to run the queries. 

1. Use uppercase for keywords

`SELECT`, `Select` and `select` will all be read by SQL. `SELECT` is best practice as it helps to easily identify which SQL keywords were used in the statement. Always capitalise keywords. 

In [ ]:
SELECT * FROM address;
-- Is preferable to
select * from address;

2. Using indentation after each keyword can improve readability of your statement, if it's quite long. Sometimes you might see columns in the `SELECT` clause more indented than preceding clauses. This can help identify when a new statement begins.

In [ ]:
SELECT address_id, 
       address, 
       address2, 
       district, 
       city_id, 
       phone
FROM 
    address
WHERE
    district = 'California'
ORDER BY
    city_id;

-- is preferable to

SELECT
address_id, 
address, 
address2, 
district, 
city_id, 
phone
FROM 
address
WHERE
district = 'California'
ORDER BY
city_id;


3. Using *snake_case* when writing your statements is the most common way to reference columns, tables and schemas. Though this may differ depending on the organisation, the idea is to be consistent with it. You will likely see *camelCase* as being the second most popular way to reference these objects. These are some of the programming language coding cases you will see:

- **snake_case**: All lowercase letters separated by an underscore
- **camelCase**: First letter of the first word is in lowercase. The first letter of all words after the first are capitalised. 
- *PascalCase*: First letter of every word is capitalised
- *kebab-case*: All words are in lowercase and separated by a hyphen

In [ ]:
SELECT film_id FROM film_actor;  /* in snake_case*/
     
-- Normally preferable to

SELECT filmId FROM filmActor;   /* in camelCase */
SELECT FilmActor FROM FilmActor;   /* in PascalCase */
SELECT film-actor FROM film-actor;  /* in kebab-case */

4. Use *aliases* where possible as it will help improve the readability of your code. You can create *aliases* for your columns in your `SELECT` statements. This will help to understand what data the column properly represents, after performing an aggregation or arithmetic operation on the column. To create an alias in your `SELECT` statement use the keyword *`AS`* after the `SELECT` followed by an alias for the column.

In [ ]:
/* This query returns the amount of rentals each customer
has made. The total amount they've paid overall and the average
of each payment. It then orders the customer by the amount they've 
paid historically in descending order */ 

SELECT customer_id, 
       SUM(amount) AS total_customer_payments,
       COUNT(rental_id) AS total_rentals_made,
       ROUND(SUM(amount) / COUNT(rental_id), 2) AS average_rental_payment
FROM
    payment
GROUP BY customer_id 
ORDER BY total_customer_payments DESC;

-- is preferable to

SELECT customer_id, 
       SUM(amount),
       COUNT(rental_id), 
       ROUND(SUM(amount) / COUNT(rental_id), 2) 
FROM
    payment
GROUP BY customer_id; 


You can see in the preferred version, it's much easier to understand what the result of our aggregations/arithmetic operation will be in the `SELECT` statement. Without the aliases SQL will name the resulting columns with the name of the first keyword in the column, which isn't very descriptive. If you had `SUM(amount)` SQL would name the resultant column `sum`, which doesn't explain what the column represents.

<h3 style="color: rgb(241, 90, 36)">Watch it in action</h3>

In [ ]:
#@title ### Run the cell to play the video{ display-mode: "form" } 
video_player.play_video("aliasing_columns.mp4")


You can see that the query is easier to understand and the data makes sense when viewing the result of the query. So try and use aliases when appropriate.

5. Try to avoid using `*` whenever possible.

The use of `*` indicates to get all columns from a table. This is known in SQL as a *wildcard*. Wildcards are special characters that represent one or more characters in a string. Specifically, `*` represents zero or more characters. Although it's a quick short-form to retrieve all columns from a table it isn't an efficient way to do so. 

Problems when using `*` to get all columns:

- Often you don't need to view all columns in the resultant set of data, `*` adds additional unneeded information
- Doesn't give you the option to reorder the columns in the resultant set of data. It might make sense for column `x` to come before column `y` in your resultant set for anyone viewing the results of your query. 
- Increases bandwidth: if your database is on another network then you're returning more data over the network than required
- Less readability in the code: it's less clear what the intended columns of interest returned by your query are
- It can be computationally quicker to only select the required columns

Sometimes if you are sure you want to get all column from the table then `*` can be used as a quick short-form. Otherwise, always identify columns if you expect someone else will be reading your code, use your best judgement here.Don't sacrifice readability with `*`, if you could make the query more understandable without it.

Let's look at an example SQL query: 

In [ ]:
/* The queries here will return the release year, description, title
of all films which actors who's first name is Nick. */

SELECT actor.first_name,
       actor.last_name,
       film.title AS film_title,
       film.release_year AS film_release_year, 
       film.description AS film_description
FROM 
    actor
JOIN 
    film_actor ON actor.actor_id = film_actor.actor_id
JOIN 
    film ON film_actor.film_id = film.film_id
WHERE 
    first_name = 'NICK';

-- is preferable to

SELECT * 
FROM 
    actor
JOIN 
    film_actor ON actor.actor_id = film_actor.actor_id
JOIN 
    film ON film_actor.film_id = film.film_id
WHERE 
    first_name = 'NICK';

By reading the first query it is much easier to understand what columns are included in the resultant data since they are named and ordered. Even if the query was much more complex by specifying the columns selected we can make a good guess at what the resultant data includes. It is much easier to understand that the first query will display all actors whose first name is `Nick`, and the movies they were in with the `release_year` and `description`. 


Reading the second query, it is not obvious which columns are the columns of interest returned by the query. These columns lack aliases and so it will be hard to decipher what the intent of the query was.

<h3 style="color: rgb(241, 90, 36)">Watch it in action</h3>


In [ ]:
#@title ### Run the cell to play the video{ display-mode: "form" } 
video_player.play_video("not_selecting_all_columns.mp4")

With the first query the data is concise and to the point, it's more obvious we're getting the details of all movies with actors named `Nick`. 

With the second query we're getting all possible columns from the `film_actor`, `film` and `actor` table since we've joined those tables together. 

Would you be able to infer the resulting data from the second query before it was run? If you did, would it be easy to extract the required information from the table?

6. Using `WHERE` to join tables instead of `JOIN`.

You can often use a `WHERE` statement in place of a `JOIN` statement though it's not generally a good idea to do so. Even though there is no performance benefit by using `JOIN` over `WHERE`, there are some other benefits:

- Seperating the `JOIN` and `WHERE` statements clearly differentiates between the filtering and the joining of the data making for more readable and easier to maintain code. 
- It is less error prone since you have to specify which table you're joining before completing the `JOIN` statement. Making it easier to understand which tables are being joined together and by which columns.

In [ ]:
SELECT actor.first_name,
       actor.last_name,
       film.title AS film_title,
       film.release_year AS film_release_year, 
       language.name as film_language, 
       film.description AS film_description
FROM 
    actor
JOIN 
    film_actor ON actor.actor_id = film_actor.actor_id
JOIN 
    film ON film_actor.film_id = film.film_id
JOIN
    language ON language.language_id = film.language_id
WHERE 
    first_name = 'Nick';

-- is preferable to 

SELECT actor.first_name,
       actor.last_name,
       film.title AS film_title,
       film.release_year AS film_release_year, 
       language.name as film_language,
       film.description AS film_description
FROM 
    actor,
    film_actor,
    film,
    language
WHERE 
    actor.actor_id = film_actor.actor_id
    AND first_name = 'Nick'
    AND film_actor.film_id = film.film_id
    AND language.language_id = film.language_id;

Reading the first query above you can see each `JOIN` has its own indented statement, helping to segregate the logic from the other joins. There is also a clear distinction between filtering the data and joining the data.

In the second statement the filtering and joining are included in the `WHERE` clause, making it harder to read and segregate the logic of the query.

7. Referencing columns in the `SELECT` statement by value

SQL gives you the ability to reference the columns in the `SELECT` statement in subsequent statements by numerical value. While it can be great to quickly reference columns using numerical values the logic can be lost when reading the statement back. Let's look at an example:


In [ ]:
SELECT rental_date,
       inventory_id, 
       customer_id,
       staff_number
FROM
    rental
WHERE 
    staff_number < 2
    AND inventory_id > 1
ORDER BY
    inventory_id, rental_date, customer_id


-- is preferable to 

SELECT rental_date,
       inventory_id, 
       customer_id,
       staff_number
FROM
    rental
WHERE 
    staff_number < 2
    AND inventory_id > 1
ORDER BY
    2, 1, 3;


There is immediate clarity in which order the column are being ordered by in the first statement.


<h2 style="color: rgb(241, 90, 36)">Conclusion</h2>

## Key Takeaways

- Employing SQL best practices will help you write more easily maintainable, readable and scalable code
- Sometimes you need to use your best judgement when a `*` is to used for instance
- Consistency is key! If you've used **snake_case** in your last three queries then continue to use **snake_case**
- It is not always required to use best practices when prototyping code but try and enforce best practices when handing code over to someone else